<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/datos/exams/Examen_Junio_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SOLUCIÓN DEL EXAMEN DE JUNIO DE 2025

In [ ]:
# Instalamos la librería statsforecast
!pip install statsforecast
# https://nixtlaverse.nixtla.io/statsforecast/docs/models/autoarima.html
# Esto es para reiniciar el kernel y así poder importar luego la librería
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00


In [1]:
#IMPORTAMOS LIBRERÍAS Y DATOS
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from statsforecast.models import AutoARIMA

# Esto hacerlo solo después de ejecutarlo una vez y saber de qué warnings se trata
import warnings
warnings.filterwarnings("ignore")

# Importamos datos de parados
df=pd.read_excel("https://github.com/otoperalias/Coyuntura/raw/refs/heads/main/clases/datos/exams/Enero25_OCUP.xlsx", index_col=0)
# Crear una columna con el periodo, para convertirla en datetime y en el index.
df['periodo']=df.index.str[:4]+"Q"+df.index.str[-1:]
df['periodo']=pd.to_datetime(df['periodo'])
df.set_index('periodo', inplace=True)

# Importamos datos de afiliados (variable X)
ss=pd.read_excel("https://github.com/otoperalias/Coyuntura/raw/refs/heads/main/clases/datos/exams/Enero25_SS.xlsx", index_col=0)
# Crear una columna con el periodo, para convertirla en datetime y en el index.
ss['periodo']=ss.index.astype(str).str[:4]+"-"+ss.index.astype(str).str[-2:]
ss['periodo']=pd.to_datetime(ss['periodo'])
ss.set_index('periodo', inplace=True)
# Pasar a trimestres (tenemos todos los meses del 4º trimestre)
ss.index=ss.index.to_period("M")
ss=ss.resample('Q',label="right").mean()
ss.index=ss.index.to_timestamp()

In [ ]:
# INDICAMOS PROVINCIA Y FILTRAMOS
name=input("Introduce el nombre de la provincia: ")
prov=df.filter(like=name)
ss_prov=ss.filter(like=name).copy()

# 1. DESESTACIONALIZACIÓN

# Calculamos la serie desestacionalizada
stl_method = sm.tsa.STL(prov.iloc[:, 0], robust=True).fit()
ocup_aj=prov.iloc[:, 0]-stl_method.seasonal

# Evolución de los ocupados
fig,ax=plt.subplots(figsize=(6,4), dpi=100)
ax.plot(prov.loc["2021":], label="Datos brutos")
ax.plot(ocup_aj.loc["2021":],label="Desestacionalizados")
ax.set_ylabel("Ocupados (miles) \n")
ax.set_xlabel("")
ax.xaxis.set_tick_params(labelsize=9)
ax.set_title("Evolución de los ocupados en "+name+"\n")
ax.legend(frameon=False, fontsize=10)
ax.grid(axis='y', alpha=0.5)
ax.spines[['top','right']].set_visible(False)
fig.text(0.1,-0.03,"Fuente: EPA (INE). Serie desestacionalizada con el método STL. Elaboración propia.", size=9)
plt.show()
print(' \n \n ')

# 2. TASA DE VARIACIÓN
oc_tc=(ocup_aj/ocup_aj.shift(1)-1 )*100

# Tasa de variación
fig,ax=plt.subplots(figsize=(6,4), dpi=100)
ax.plot(oc_tc.loc["2021":])
ax.set_ylabel("Tasa de variación (%) \n")
ax.set_xlabel("")
ax.xaxis.set_tick_params(labelsize=9)
ax.set_title("Variación intertrimestral de los ocupados en "+name)
ax.grid(axis='y', alpha=0.5)
ax.spines[['top','right']].set_visible(False)
fig.text(0.1,-0.03,"Fuente: EPA (INE). Serie desestacionalizada con el método STL. Elaboración propia.", size=9)
plt.show()
print(' \n \n ')

# 3. PREDICCIÓN
# PREDICCIÓN MODELO SARIMA
smodel = pm.auto_arima(prov, start_p=0, start_q=0, test='adf', max_p=3, max_q=3, m=4,
                         start_P=0, seasonal=True, error_action='ignore',
                         suppress_warnings=True, stepwise=True)
fitted, confint = smodel.predict(n_periods=1, return_conf_int=True)
index_of_fc = pd.date_range(start="2024-10-01",end="2024-10-01", freq='QS')
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# PREDICCIÓN MODELO SARIMAX
smodel = pm.auto_arima(prov["2009":], start_p=0, start_q=0, X=ss_prov[:-1], test='adf', max_p=3, max_q=3, m=4,
                         start_P=0, seasonal=True, error_action='ignore',
                         suppress_warnings=True, stepwise=True)
fitted2, confint2 = smodel.predict(n_periods=1, X=ss_prov[-1:], return_conf_int=True)
lower_series2 = pd.Series(confint[:, 0], index=index_of_fc)
upper_series2 = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
fig,(ax1,ax2)=plt.subplots(1,2, sharey=True, figsize=(10,4.25), dpi=100)
  #SARIMA
ax1.plot(prov.loc["2021":], color="tab:blue")
ax1.plot(fitted, color='r',marker='.')
ax1.plot(lower_series, color='r', marker='_')
ax1.plot(upper_series, color='r',marker='_')
ax1.set_title("Predicción 1: SARIMA \n", size=10, pad=20)
ax1.set_ylabel("Ocupados (miles) \n")
ax1.grid(axis='y')
ax1.xaxis.set_tick_params(labelsize=8, labelrotation=45)
ax1.spines[['top','right']].set_visible(False)
ax1.text(fitted.index[0],fitted[0]*1.01,round(fitted[0],1), fontsize=9)
  #SARIMAX
ax2.plot(prov.loc["2021":], color="tab:blue")
ax2.plot(fitted2, color='r',marker='.')
ax2.plot(lower_series2, color='r', marker='_')
ax2.plot(upper_series2, color='r',marker='_')
ax2.set_title("Predicción 2: SARIMAX \n", size=10, pad=20)
ax2.set_ylabel("Ocupados (miles) \n")
ax2.grid(axis='y')
ax2.xaxis.set_tick_params(labelsize=8, labelrotation=45)
ax2.spines[['top','right']].set_visible(False)
ax2.text(fitted2.index[0],fitted2[0]*1.01,round(fitted2[0],1), fontsize=9)
  # Notas
fig.text(0.3,1.1,"Predicción de ocupados en "+name+" en el 4º tr 2024", size=12)
fig.text(0.1,-0.08,"Fuente: EPA (INE) y INSS. Elaboración propia. La predicción 1 se ha realizado con el modelo SARIMA y la predicción 2 con el SARIMAX,\n \
con la variable de afiliados medios totales como variable exógena.", size=9)
plt.show()

